In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [5]:
def get_water_rights_data(**kwargs):
    """
    Query the Utah Water Rights calculation service with keyword arguments.
    
    Returns:
        str: Response text from the API if successful, None otherwise
    """
    # Base URL
    base_url = "https://maps.waterrights.utah.gov/Py/calculate.py"
    
    # Default parameters
    default_params = {
        "PageId": "Evaluate PODS",
        "LOGINFLAG": "false",
        "Units": "plss",
        "UTMx": "",
        "datumutm": "NAD83",
        "utmcorner": "closest",
        "UTMy": "",
        "plsns": "N",
        "plsnsdist": "100",
        "plsew": "E",
        "plsewdist": "500",
        "plscorner": "NE",
        "plsection": "8",
        "plstown": "3N",
        "plsrange": "4E",
        "plsbm": "SL",
        "LLy": "",
        "datumLL": "NAD83",
        "LLcorner": "closest",
        "LLx": "",
        "SPx": "",
        "datumSP": "NAD83",
        "zoneSP": "4301.0",
        "SPcorner": "closest",
        "SPy": "",
        "batchdatumutm": "NAD83",
        "utm_pnts": ""
    }
    
    # Update default parameters with any provided kwargs
    params = {**default_params, **kwargs}
    
    try:
        # Make the GET request
        response = requests.get(base_url, params=params)
        
        # Raise an exception for bad status codes
        response.raise_for_status()
        
        # Return the response content
        return response.text
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None




In [12]:
def parse_location(location):
    # Split the location string into components
    components = location.split()
    
    # Extract key components from the location
    try:
        ns_distance = components[1]  # e.g., 190
        ew_distance = components[3]  # e.g., 1660
        section = components[5]           # e.g., 25
        township = components[6]          # e.g., S22
        range_ = components[7]            # e.g., E1

        # Reformat township and range
        township = f"{township[1:]}{township[0]}"  # Converts "S22" to "22S"
        range_ = f"{range_[1:]}{range_[0]}"        # Converts "E1" to "1E"

        # Create the parameter dictionary
        result = get_water_rights_data(
            plsns= components[0],              # North or South (N/S)
            plsnsdist= ns_distance,            # Distance in the North-South direction
            plsew= components[2],              # East or West (E/W)
            plsewdist= ew_distance,            # Distance in the East-West direction
            plsection= section,                # Section number
            plstown= township,                 # Township value
            plsrange= range_,                  # Range value
        )
        return result
    except (IndexError, ValueError):
        print("Invalid location format")
        return None






In [14]:
from bs4 import BeautifulSoup


from bs4 import BeautifulSoup
import pandas as pd

def extract_coordinates_from_html(html_content):
    """
    Parses HTML content to extract UTM NAD83, Latitude/Longitude WGS84, 
    splits them into separate values, and returns them as a Pandas Series.

    Parameters:
        html_content (str): HTML content as a string.

    Returns:
        pd.Series: A pandas Series with extracted UTM NAD83 and Latitude/Longitude as separate values.
    """
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Dictionary to store extracted data
    data = {}

    # Find table rows and extract relevant data
    table_rows = soup.find_all("tr")
    for row in table_rows:
        cells = row.find_all("td")
        if len(cells) >= 2:
            label = cells[0].get_text(strip=True)
            value = cells[1].get_text(strip=True)
            if "UTM NAD83" in label:
                # Split UTM NAD83 into Easting and Northing
                easting, northing = value.split(',')
                data["UTM_NAD83_Easting"] = float(easting)
                data["UTM_NAD83_Northing"] = float(northing)
            elif "Lat/Long WGS84" in label or "Geographic WGS84" in label:
                # Split Latitude/Longitude into separate values
                latitude, longitude = value.split(',')
                data["Latitude_WGS84"] = float(latitude)
                data["Longitude_WGS84"] = float(longitude)

    # Convert dictionary to Pandas Series
    return pd.Series(data)




UTM_NAD83_Easting     4.353985e+05
UTM_NAD83_Northing    4.303061e+06
Latitude_WGS84        3.887404e+01
Longitude_WGS84      -1.117447e+02
dtype: float64
UTM NAD83: 435398.450,4303061.416
Latitude and Longitude (WGS84): 38.8740376918, -111.744735561


In [ ]:
# Example location from the table
location_example = "N 190 W 1660 SE 25 S22 E1"

# Parse the location
html = parse_location(location_example)

# Example usage

result_series = extract_coordinates_from_html(html)

# Display the result
print(result_series)

print(f"UTM NAD83: {utm_nad83}")
print(f"Latitude and Longitude (WGS84): {latitude_longitude}")

In [ ]:
serieses = {}

for address in df['PLSS'].values:
    try:
        html = parse_location(location_example)
        result_series = extract_coordinates_from_html(html)
        result_series['PLSS'] = address
        serieses[address] = result_series
    except Exception as e:
        print(e)
        pass
    

new_coordinates = pd.concat(serieses)